In [12]:
import numpy as np
from math import floor, ceil
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, mean_squared_error
from keras.utils import np_utils
#import sys, os
#sys.path.append(os.path.abspath('../../src/neural_network'))
#sys.path.append('../../src/neural_network')
from neural_network import NeuralNetwork
from fc_layer import FCLayer
from activation_layer import ActivationLayer
from activation_functions import sigmoid, sigmoid_derivative, identity, identity_derivative, tanh, tanh_derivative, relu, relu_derivative
from loss_functions import mse, mse_derivative
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler

In [ ]:
hyperparameters = {
    "type": "Regression",
    #"number_of_layers": [3, 4, 5],
    #"hidden_layer_sizes": [2, 1, 1, 1, 1],
    "output_layer_size": 1,
    #"activation_function": "sigmoid",
    #"learning_rate": 0.1,
    #"epochs": 100,
    #"batch_size": 1,
    #"loss_function": "mean_squared_error",
    #"optimizer": "adam",
}

In [13]:
X, Y = load_wine(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
#Normalizing data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(178, 13) (178,)
(142, 13) (36, 13) (142, 3) (36, 3)


In [14]:
class NetworkGenerator:
    def __init__(self, X_train, y_train, X_test, y_test, type):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.type = type
        self.input_layer_size = X_train.shape[1]
        if(self.type == "Regression"):
            self.output_layer_size = 1
            self.activation_function = relu
            self.activation_function_derivative = relu_derivative
        else: 
            self.output_layer_size = y_train.shape[1]
            self.activation_function = tanh
            self.activation_function_derivative = tanh_derivative
        self.middle_layer_size = ceil((2*(self.input_layer_size + self.output_layer_size))/3)
        
#interpolating the hidden layer sizes
    def __interpolate_hidden_layer_sizes(self, middle_layer_size, number_of_layers):
        hidden_layer_sizes = [0 for x in range(number_of_layers)]
        layers_before_middle = floor(number_of_layers/2) #2
        layers_after_middle = ceil(number_of_layers/2) - 1 #2
        hidden_layer_sizes[layers_before_middle] = middle_layer_size
        index = layers_before_middle - 1
        for i in range(layers_before_middle, 0, -1):
            hidden_layer_sizes[index] = ceil(2*(self.input_layer_size + hidden_layer_sizes[i])/3)
            index -= 1
        
        index = layers_before_middle + 1
        for i in range(layers_before_middle, number_of_layers - 1, 1):
            hidden_layer_sizes[index] = ceil(2*(hidden_layer_sizes[i] + self.output_layer_size)/3)
            index += 1
        return hidden_layer_sizes

    def __create_network(self, middle_layer_size, number_of_layers, epochs, alpha):
        network = NeuralNetwork()
        hidden_layer_sizes = self.__interpolate_hidden_layer_sizes(middle_layer_size, number_of_layers)
        hidden_layer_sizes.insert(0, self.input_layer_size)
        hidden_layer_sizes.append(self.output_layer_size)
        for i in range(len(hidden_layer_sizes) - 1):
            network.add(FCLayer(hidden_layer_sizes[i], hidden_layer_sizes[i + 1]))
            if i < len(hidden_layer_sizes) - 1:
                network.add(ActivationLayer(self.activation_function, self.activation_function_derivative))
        if self.type == "Classification":
            network.add(ActivationLayer(sigmoid, sigmoid_derivative))
            network.use(mse, mse_derivative)  #TODO: change loss function
        else: 
            network.add(ActivationLayer(identity, identity_derivative))
            network.use(mse, mse_derivative)
        network.fit(self.X_train, self.y_train, epochs=epochs, learning_rate=alpha)
        return network
            
    def __network_generator(self, config, reporter):
        network = self.__create_network(config["middle_layer_size"], config["number_of_layers"], config["epochs"], config["alpha"])
        y_pred = np.array(network.predict(self.X_test))
        if self.type == "Classification":
            reporter(config, mean_accuracy = accuracy_score(self.y_test, y_pred.round()))
        else:
            reporter(config, mean_loss = mean_squared_error(self.y_test, y_pred))
        
    def __train_network(self):
        scheduler = AsyncHyperBandScheduler()
        if self.type == "Classification":
            res = tune.run(
            self.__network_generator,
            name="my_exp",
            metric="mean_accuracy",
            stop={"mean_accuracy": 0.8},
            mode="max",
            scheduler=scheduler,
            config={
                "number_of_layers": tune.grid_search([3, 4]),#, 5]),
                "middle_layer_size": tune.grid_search([8]),#,32, 64, 128]),
                "epochs": tune.grid_search([500, 1000]),#, 2000, 3000, 4000, 5000]),
                "alpha": tune.grid_search([0.001, 0.01, 0.1, 1]),
            },
            )
            results = {k: v for k, v in sorted(res.results.items(), key=lambda item: (item[1]["mean_accuracy"], item[1]["time_this_iter_s"]), reverse=True)}
            highest_accuracy = results[list(results.keys())[0]]["mean_accuracy"]
            index = -1
            for k, v in results.items():
                if v["mean_accuracy"] != highest_accuracy:
                    return list(results.values())[index]
                index += 1
                
        else:
            res = tune.run(
            self.__network_generator,
            name="my_exp",
            metric="mean_loss",
            stop={"mean_loss": 0.2},
            mode="min",
            scheduler=scheduler,
            config={
                "number_of_layers": tune.grid_search([3, 4, 5]),
                "middle_layer_size": tune.grid_search([8, 16, 32, 64, 128]),
                "epochs": tune.grid_search([500, 1000, 2000, 3000, 4000, 5000]),
                "alpha": tune.grid_search([0.001, 0.01, 0.1, 1]),
            },
            )
            results = {k: v for k, v in sorted(res.results.items(), key=lambda item: (item[1]["mean_loss"], item[1]["time_this_iter_s"]))}
            return list(results.values())[0]
        
    def get_best_network(self):
        best_result = self.__train_network()
        accuracy = best_result["mean_accuracy"] if self.type == "Classification" else best_result["mean_loss"]
        config = best_result["config"]
        network = self.__create_network(config["middle_layer_size"], config["number_of_layers"], config["epochs"], config["alpha"])
        return network, accuracy, config

In [15]:
network_generator = NetworkGenerator(X_train, y_train, X_test, y_test, "Classification")
network = network_generator.get_best_network()
#sizes = network_generator.interpolate_hidden_layer_sizes(128, 5)
#print(sizes)

2022-05-15 21:37:49,302	INFO registry.py:66 -- Detected unknown callable for trainable. Converting to class.


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_80ca4_00000,PENDING,,0.001,500,8,3
__network_generator_80ca4_00001,PENDING,,0.01,500,8,3
__network_generator_80ca4_00002,PENDING,,0.1,500,8,3
__network_generator_80ca4_00003,PENDING,,1,500,8,3
__network_generator_80ca4_00004,PENDING,,0.001,1000,8,3
__network_generator_80ca4_00005,PENDING,,0.01,1000,8,3
__network_generator_80ca4_00006,PENDING,,0.1,1000,8,3
__network_generator_80ca4_00007,PENDING,,1,1000,8,3
__network_generator_80ca4_00008,PENDING,,0.001,500,8,4
__network_generator_80ca4_00009,PENDING,,0.01,500,8,4


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_80ca4_00000,PENDING,,0.001,500,8,3
__network_generator_80ca4_00001,PENDING,,0.01,500,8,3
__network_generator_80ca4_00002,PENDING,,0.1,500,8,3
__network_generator_80ca4_00003,PENDING,,1,500,8,3
__network_generator_80ca4_00004,PENDING,,0.001,1000,8,3
__network_generator_80ca4_00005,PENDING,,0.01,1000,8,3
__network_generator_80ca4_00006,PENDING,,0.1,1000,8,3
__network_generator_80ca4_00007,PENDING,,1,1000,8,3
__network_generator_80ca4_00008,PENDING,,0.001,500,8,4
__network_generator_80ca4_00009,PENDING,,0.01,500,8,4


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers
__network_generator_80ca4_00000,RUNNING,,0.001,500,8,3
__network_generator_80ca4_00001,RUNNING,,0.01,500,8,3
__network_generator_80ca4_00002,RUNNING,,0.1,500,8,3
__network_generator_80ca4_00003,RUNNING,,1,500,8,3
__network_generator_80ca4_00004,RUNNING,,0.001,1000,8,3
__network_generator_80ca4_00005,RUNNING,,0.01,1000,8,3
__network_generator_80ca4_00006,RUNNING,,0.1,1000,8,3
__network_generator_80ca4_00007,RUNNING,,1,1000,8,3
__network_generator_80ca4_00008,PENDING,,0.001,500,8,4
__network_generator_80ca4_00009,PENDING,,0.01,500,8,4


Result for __network_generator_80ca4_00000:
  _metric:
    alpha: 0.001
    epochs: 500
    middle_layer_size: 8
    number_of_layers: 3
  date: 2022-05-15_21-38-37
  done: true
  experiment_id: 4e63cb8fc58648dd80023325cf22a320
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.9444444444444444
  node_ip: 192.168.1.105
  pid: 9768
  time_since_restore: 34.0319550037384
  time_this_iter_s: 34.0319550037384
  time_total_s: 34.0319550037384
  timestamp: 1652643517
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 80ca4_00000
  


...

Result for __network_generator_80ca4_00015:
  _metric:
    alpha: 1
    epochs: 1000
    middle_layer_size: 8
    number_of_layers: 4
  date: 2022-05-15_21-40-56
  done: true
  experiment_id: d45e3b8660614d11a83e01e5427f9c85
  hostname: Ghallab
  iterations_since_restore: 1
  mean_accuracy: 0.9722222222222222
  node_ip: 192.168.1.105
  pid: 10852
  time_since_restore: 67.54429888725281
  time_this_iter_s: 67.54429888725281
  time_total_s: 67.54429888725281
  timestamp: 1652643656
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 80ca4_00015
  


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers,acc,iter,total time (s)
__network_generator_80ca4_00000,TERMINATED,,0.001,500,8,3,0.944444,1,34.032
__network_generator_80ca4_00001,TERMINATED,,0.01,500,8,3,0.944444,1,40.5585
__network_generator_80ca4_00002,TERMINATED,,0.1,500,8,3,0.972222,1,39.8424
__network_generator_80ca4_00003,TERMINATED,,1,500,8,3,0.972222,1,40.0548
__network_generator_80ca4_00004,TERMINATED,,0.001,1000,8,3,0.944444,1,79.7865
__network_generator_80ca4_00005,TERMINATED,,0.01,1000,8,3,0.972222,1,79.1084
__network_generator_80ca4_00006,TERMINATED,,0.1,1000,8,3,0.972222,1,79.3028
__network_generator_80ca4_00007,TERMINATED,,1,1000,8,3,0.944444,1,72.8411
__network_generator_80ca4_00008,TERMINATED,,0.001,500,8,4,0.944444,1,45.0026
__network_generator_80ca4_00009,TERMINATED,,0.01,500,8,4,0.944444,1,47.3054


(pid=10852) Windows fatal exception: access violation
(pid=10852) 


Trial name,status,loc,alpha,epochs,middle_layer_size,number_of_layers,acc,iter,total time (s)
__network_generator_80ca4_00000,TERMINATED,,0.001,500,8,3,0.944444,1,34.032
__network_generator_80ca4_00001,TERMINATED,,0.01,500,8,3,0.944444,1,40.5585
__network_generator_80ca4_00002,TERMINATED,,0.1,500,8,3,0.972222,1,39.8424
__network_generator_80ca4_00003,TERMINATED,,1,500,8,3,0.972222,1,40.0548
__network_generator_80ca4_00004,TERMINATED,,0.001,1000,8,3,0.944444,1,79.7865
__network_generator_80ca4_00005,TERMINATED,,0.01,1000,8,3,0.972222,1,79.1084
__network_generator_80ca4_00006,TERMINATED,,0.1,1000,8,3,0.972222,1,79.3028
__network_generator_80ca4_00007,TERMINATED,,1,1000,8,3,0.944444,1,72.8411
__network_generator_80ca4_00008,TERMINATED,,0.001,500,8,4,0.944444,1,45.0026
__network_generator_80ca4_00009,TERMINATED,,0.01,500,8,4,0.944444,1,47.3054


2022-05-15 21:40:56,816	INFO tune.py:561 -- Total run time: 187.52 seconds (187.33 seconds for the tuning loop).


In [16]:
y_pred = np.array(network[0].predict(X_test))
print(accuracy_score(y_test, y_pred.round()), network[1], network[2])

0.9722222222222222 0.9722222222222222 {'number_of_layers': 3, 'middle_layer_size': 8, 'epochs': 500, 'alpha': 0.1}


(pid=None) E0516 17:14:48.271000000 15352 external/com_github_grpc_grpc/src/core/ext/transport/chttp2/transport/chttp2_transport.cc:2876] ipv4:192.168.1.105:64222: Keepalive watchdog fired. Closing transport.
